In [ ]:
import urllib
import gromacs as gmx
import nglview as nv
import mdtraj as md

In [ ]:
pdb='1L2Y'
urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb}.pdb',f'{pdb}.pdb')

In [ ]:
gmx.pdb2gmx(f=pdb+'.pdb',o=pdb+'.gro',water='tip3p',ff='amber99',p=pdb+'.top',ignh=True)

In [ ]:
gmx.editconf(f=pdb+'.gro',o=pdb+'-box.gro',c=True,d='1.5', bt='dodecahedron')

In [ ]:
gmx.solvate(cp=pdb+'-box.gro',cs='spc216.gro',o=pdb+'-solv.gro',p=pdb+'.top')

In [ ]:
with open('ions.mdp','w') as ions:
    ions.write("""\
; ions.mdp - used as input into grompp to generate ions.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme	= Verlet    ; Buffered neighbor searching 
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = cutoff    ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions
""")
    
gmx.grompp(f='ions.mdp',c=pdb+'-solv.gro',p=pdb+'.top',o='ions.tpr')

In [ ]:
gmx.select(s=pdb+'-solv.gro',on='solv.ndx',select='SOL')

In [ ]:
gmx.genion(s='ions.tpr',n='solv.ndx',o=pdb+'-ions.gro',p=pdb+'.top',pname='NA',nname='CL',neutral=True)

In [ ]:
with open('minim.mdp','w') as m:
    m.write("""\
; minim.mdp - used as input into grompp to generate em.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions

nstxout                 = 50         
nstvout                 = 0        
nstfout                 = 0
nstenergy               = 50         
""")
    
gmx.grompp(f='minim.mdp',c=pdb+'-ions.gro',p=pdb+'.top',o='em.tpr')

In [ ]:
mdrun=gmx.MDrunnerK8s()

In [ ]:
ompthreads=2
mpiranks=1
mdrun.run(pre={'cores':ompthreads*mpiranks,'gpus':1}, mdrunargs={'deffnm':'em','ntomp':ompthreads},ncores=mpiranks)

In [ ]:
gmx.select(s=pdb+'-ions.gro',on='protein.ndx',select='Protein')

In [ ]:
gmx.trjconv(s=pdb+'-ions.gro',f='em.trr',n='protein.ndx',o='em-protein.xtc')

In [ ]:
tr=md.load('em-protein.xtc',top=pdb+'.gro')
nv.show_mdtraj(tr)